In [4]:
from bs4 import BeautifulSoup
import requests
import sys

In [5]:
web = 'https://fbref.com/en/comps/23/Eredivisie-Stats'
res = requests.get(web)

if res.status_code == 200:
    soup = BeautifulSoup(res.text)
    standings_table = soup.select('table.stats_table')[0]
    links = standings_table.find_all('a')
    links = [l.get("href") for l in links]
    links = [l for l in links if '/squads/' in l]

else:
    # Print an error message if the request was not successful
    print(f"HTTP request failed with status code: {res.status_code}")
    print(f"Inner message: {res.reason}")
    retry_after = res.headers.get('Retry-After')
    if retry_after:
        print(f"Retry-After: {retry_after}")
    else:
        print("Retry-After header not present in the response.")
    sys.exit()  # or return if you are inside a function

In [6]:
links

['/en/squads/e334d850/PSV-Eindhoven-Stats',
 '/en/squads/fb4ca611/Feyenoord-Stats',
 '/en/squads/a1f721d3/Twente-Stats',
 '/en/squads/3986b791/AZ-Alkmaar-Stats',
 '/en/squads/19c3f8c4/Ajax-Stats',
 '/en/squads/fc629994/NEC-Nijmegen-Stats',
 '/en/squads/146a68ce/Sparta-Rotterdam-Stats',
 '/en/squads/e33d6108/Go-Ahead-Eagles-Stats',
 '/en/squads/740cb7d4/Excelsior-Stats',
 '/en/squads/e3db180b/Zwolle-Stats',
 '/en/squads/193ff7aa/Heerenveen-Stats',
 '/en/squads/bd08295c/Fortuna-Sittard-Stats',
 '/en/squads/2b41acb5/Almere-City-Stats',
 '/en/squads/c882b88e/Heracles-Almelo-Stats',
 '/en/squads/2a428619/Utrecht-Stats',
 '/en/squads/bb14adb3/RKC-Waalwijk-Stats',
 '/en/squads/8783f4ee/Volendam-Stats',
 '/en/squads/209d7fa2/Vitesse-Stats']

In [7]:
team_urls = [f"https://fbref.com{l}" for l in links]

In [8]:
data = requests.get(team_urls[0])

In [ ]:
import pandas as pd
matches = pd.read_html(data.text, match="Scores & Fixtures")[0]

In [10]:
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/shooting/' in l]

In [11]:
data = requests.get(f"https://fbref.com{links[0]}")

In [ ]:
shooting = pd.read_html(data.text, match="Shooting")[0]

In [13]:
shooting.head()

For PSV Eindhoven                                                          \
               Date   Time          Comp                   Round  Day Venue   
0        2023-08-08  20:30  Champions Lg  Third qualifying round  Tue  Home   
1        2023-08-12  16:30    Eredivisie             Matchweek 1  Sat  Home   
2        2023-08-15  20:30  Champions Lg  Third qualifying round  Tue  Away   
3        2023-08-19  18:45    Eredivisie             Matchweek 2  Sat  Away   
4        2023-08-22  20:00  Champions Lg          Play-off round  Tue  Away   

                                   ... Standard               Expected       \
  Result   GF   GA       Opponent  ...     Dist   FK PK PKatt       xG npxG   
0      W  4.0  1.0  at Sturm Graz  ...      NaN  NaN  0     0      NaN  NaN   
1      W  2.0  0.0        Utrecht  ...     19.9  1.0  0     0      2.6  2.6   
2      W  3.0  1.0  at Sturm Graz  ...      NaN  NaN  1     1      NaN  NaN   
3      W  3.0  1.0        Vitesse  ...     18.6  0.0  1     1      2.3  1.5   
4      D  2.0  2.0    sct Rangers  ...      NaN  NaN  0     0      NaN  NaN   

                       Unnamed: 25_level_0  
  npxG/Sh G-xG np:G-xG        Match Report  
0     NaN  NaN     NaN        Match Report  
1    0.12 -0.6    -0.6        Match Report  
2     NaN  NaN     NaN        Match Report  
3    0.08  0.7     0.5        Match Report  
4     NaN  NaN     NaN        Match Report  

[5 rows x 26 columns]

In [14]:
shooting.columns = shooting.columns.droplevel()

In [15]:
team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")

In [16]:
team_data.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Formation,Referee,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt
0,2023-08-08,20:30,Champions Lg,Third qualifying round,Tue,Home,W,4.0,1.0,at Sturm Graz,...,4-3-3,Radu Petrescu,Match Report,Leg 1 of 2,NaN,NaN,NaN,NaN,0,0
1,2023-08-12,16:30,Eredivisie,Matchweek 1,Sat,Home,W,2.0,0.0,Utrecht,...,4-3-3,Serdar Gözübüyük,Match Report,NaN,23.0,8.0,19.9,1.0,0,0
2,2023-08-15,20:30,Champions Lg,Third qualifying round,Tue,Away,W,3.0,1.0,at Sturm Graz,...,4-3-3,Srđan Jovanović,Match Report,Leg 2 of 2; PSV Eindhoven won,NaN,NaN,NaN,NaN,1,1
3,2023-08-19,18:45,Eredivisie,Matchweek 2,Sat,Away,W,3.0,1.0,Vitesse,...,4-2-3-1,Bas Nijhuis,Match Report,NaN,18.0,6.0,18.6,0.0,1,1
4,2023-08-22,20:00,Champions Lg,Play-off round,Tue,Away,D,2.0,2.0,sct Rangers,...,4-3-3,Clément Turpin,Match Report,Leg 1 of 2,19.0,5.0,NaN,NaN,0,0


In [17]:
years = list(range(2024, 2023, -1))
all_matches = []

In [18]:
standings_url = "https://fbref.com/en/comps/23/Eredivisie-Stats"

In [ ]:
import time
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == "Eredivisie"]
        
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(1)

In [20]:
len(all_matches)

18

In [21]:
match_df = pd.concat(all_matches)

In [22]:
match_df.columns = [c.lower() for c in match_df.columns]

In [23]:
match_df

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
1,2023-08-12,16:30,Eredivisie,Matchweek 1,Sat,Home,W,2.0,0.0,Utrecht,...,Match Report,NaN,23.0,8.0,19.9,1.0,0,0,2024,PSV Eindhoven
3,2023-08-19,18:45,Eredivisie,Matchweek 2,Sat,Away,W,3.0,1.0,Vitesse,...,Match Report,NaN,18.0,6.0,18.6,0.0,1,1,2024,PSV Eindhoven
6,2023-09-02,20:00,Eredivisie,Matchweek 4,Sat,Away,W,4.0,0.0,RKC Waalwijk,...,Match Report,NaN,22.0,9.0,17.5,0.0,0,0,2024,PSV Eindhoven
7,2023-09-16,20:00,Eredivisie,Matchweek 5,Sat,Home,W,4.0,0.0,NEC Nijmegen,...,Match Report,NaN,26.0,9.0,15.9,0.0,2,2,2024,PSV Eindhoven
9,2023-09-23,20:00,Eredivisie,Matchweek 6,Sat,Away,W,4.0,0.0,Almere City,...,Match Report,NaN,20.0,9.0,16.3,0.0,0,0,2024,PSV Eindhoven
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12,2023-11-25,21:00,Eredivisie,Matchweek 13,Sat,Away,L,0.0,5.0,Ajax,...,Match Report,NaN,9.0,4.0,19.6,0.0,0,0,2024,Vitesse
13,2023-12-02,20:00,Eredivisie,Matchweek 14,Sat,Away,L,1.0,3.0,Fortuna Sittard,...,Match Report,NaN,8.0,3.0,19.8,0.0,0,0,2024,Vitesse
14,2023-12-10,16:45,Eredivisie,Matchweek 15,Sun,Home,W,2.0,0.0,Heracles Almelo,...,Match Report,NaN,12.0,2.0,15.1,0.0,0,0,2024,Vitesse
15,2023-12-17,12:15,Eredivisie,Matchweek 16,Sun,Away,L,0.0,5.0,Almere City,...,Match Report,NaN,7.0,4.0,18.9,0.0,0,0,2024,Vitesse


In [24]:
match_df.to_csv("matches.csv")